In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

#### 1. Doc Load & chunking

#### NOte. "Groq offers Open-Source LLMS Executed on cloud Platform for faster inference compared to local downloaded Ollama verVersionssaions"

In [37]:
from langchain_groq import ChatGroq

llm1=ChatGroq(api_key=os.getenv("grok_api_key"),model="gemma2-9b-it")
llm1

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021B0898A5D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021B0AA5C4D0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [48]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded onlin

In [49]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
LLMGraphTransformer_obj=LLMGraphTransformer(llm=llm1)

In [50]:
graph_docs=LLMGraphTransformer_obj.convert_to_graph_documents(documents)

In [73]:
graph_docs

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Maye', type='Person', properties={}), Node(id='Errol Musk', type='Person', properties={}), Node(id='Kimbal Musk', type='Person', properties={}), Node(id='Spacex', type='Company', properties={}), Node(id='Tesla, Inc.', type='Company', properties={}), Node(id='X Corp.', type='Company', properties={}), Node(id='Twitter', type='Company', properties={}), Node(id='The Boring Company', type='Company', properties={}), Node(id='Xai', type='Company', properties={}), Node(id='Neuralink', type='Company', properties={}), Node(id='Openai', type='Company', properties={}), Node(id='University Of Pretoria', type='University', properties={}), Node(id="Queen'S University At Kingston", type='University', properties={}), Node(id='University Of Pennsylvania', type='University', properties={}), Node(id='Stanford University', type='University', properties={}), Node(id='Zip2', type='Company', properties={})], relationships

**NOTE2:-Following core entitIes(noDES & RELATIONSHIPS) Represent the schema in which Above data Chunnks is stored in NEO4J_GRAPHDB**

In [56]:
graph_docs[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Maye', type='Person', properties={}),
 Node(id='Errol Musk', type='Person', properties={}),
 Node(id='Kimbal Musk', type='Person', properties={}),
 Node(id='Spacex', type='Company', properties={}),
 Node(id='Tesla, Inc.', type='Company', properties={}),
 Node(id='X Corp.', type='Company', properties={}),
 Node(id='Twitter', type='Company', properties={}),
 Node(id='The Boring Company', type='Company', properties={}),
 Node(id='Xai', type='Company', properties={}),
 Node(id='Neuralink', type='Company', properties={}),
 Node(id='Openai', type='Company', properties={}),
 Node(id='University Of Pretoria', type='University', properties={}),
 Node(id="Queen'S University At Kingston", type='University', properties={}),
 Node(id='University Of Pennsylvania', type='University', properties={}),
 Node(id='Stanford University', type='University', properties={}),
 Node(id='Zip2', type='Company', properties={})]

In [57]:
graph_docs[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Company', properties={}), type='WORKS_FOR', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla, Inc.', type='Company', properties={}), type='WORKS_FOR', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X Corp.', type='Company', properties={}), type='OWNS', properties={}),
 Relationship(source=Node(id='X Corp.', type='Company', properties={}), target=Node(id='Twitter', type='Company', properties={}), type='FORMER_NAME', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='The Boring Company', type='Company', properties={}), type='FOUNDED', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Xai', type='Company', properties={}), type='FOUN

### 2. Loading csv data into Neo4J GraphDB

In [77]:
from langchain_community.graphs import Neo4jGraph


In [78]:
Neo4Jgraph_obj_conn=Neo4jGraph(url=os.getenv("NEO4J_URI"),
                      username=os.getenv("NEO4J_USERNAME"),
                      password=os.getenv("NEO4J_PASSWORD"),
                      database=os.getenv("NEO4J_DATABASE"))

In [79]:
Neo4Jgraph_obj_conn

In [80]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [81]:
Neo4Jgraph_obj_conn.get_schema

'Node properties:\nperson {name: STRING, age: STRING, gender: STRING}\nperson2 {name: STRING, age: STRING, name1: STRING, nae: STRING}\nmovie {name: STRING, genre: STRING}\nMovie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}\nPerson {name: STRING}\nGenre {name: STRING}\nRelationship properties:\n\nThe relationships:\n(:Movie)-[:IN_GENRE]->(:Genre)\n(:Person)-[:DIRECTED]->(:Movie)\n(:Person)-[:ACTED_IN]->(:Movie)'

In [82]:
Neo4Jgraph_obj_conn.query(movie_query)

[]

In [83]:
Neo4Jgraph_obj_conn.refresh_schema()
print(Neo4Jgraph_obj_conn.get_schema)

Node properties:
person {name: STRING, age: STRING, gender: STRING}
person2 {name: STRING, age: STRING, name1: STRING, nae: STRING}
movie {name: STRING, genre: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


### 3. Querying from NEO4J GDB using qa chain

In [87]:
from langchain.chains import GraphCypherQAChain
graphchain_obj=GraphCypherQAChain.from_llm(llm=llm1,graph=Neo4Jgraph_obj_conn,verbose=True,allow_dangerous_requests=True)

In [91]:
graphchain_obj

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000021B0CBB4F50>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021B0898A5D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021B0AA5C4D0>, model_name=

In [93]:
response=graphchain_obj.invoke({"query":"Who was the director of the moview GoldenEye"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:DIRECTED]-(p:Person)
RETURN p.name
Full Context:
[{'p.name': 'Martin Campbell'}]

> Finished chain.


{'query': 'Who was the director of the moview GoldenEye',
 'result': 'Martin Campbell  \n'}

In [ ]:
"""******************************the  end********************************************"""